In [ ]:
from main import download_data
CRYPTOS = ['BTC', 'ETH', 'EOS', 'LTC']
download_data(CRYPTOS, currency='USD')

In [1]:
from datetime import datetime, timedelta
from typing import List, Tuple
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
import tools as T

In [2]:
spark = SparkSession.builder.appName("Crypto Data").getOrCreate()
df = spark.read.csv("/home/jovyan/data/data_raw.csv", inferSchema=True, encoding='utf-8', header=True)
df.cache()
df.printSchema()

root
 |-- close: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- open: double (nullable = true)
 |-- time: timestamp (nullable = true)
 |-- volumefrom: double (nullable = true)
 |-- volumeto: double (nullable = true)
 |-- symbol: string (nullable = true)



In [3]:
df.show(3, truncate=False)

+-------+-------+-------+-------+-------------------+----------+--------+------+
|close  |high   |low    |open   |time               |volumefrom|volumeto|symbol|
+-------+-------+-------+-------+-------------------+----------+--------+------+
|0.06238|0.06238|0.06238|0.06238|2010-09-05 00:00:00|0.0       |0.0     |BTC   |
|0.06238|0.06238|0.06238|0.06238|2010-09-05 01:00:00|0.0       |0.0     |BTC   |
|0.06238|0.06238|0.06238|0.06238|2010-09-05 02:00:00|0.0       |0.0     |BTC   |
+-------+-------+-------+-------+-------------------+----------+--------+------+
only showing top 3 rows



In [4]:
# new colums for the new df
DELTA_MATCH = {
    'price': {
        'price-0h-avg': 0,
        'price-1h-avg': -1,
        'price-2h-avg': -2,
        'price-4h-avg': -4,
        'price-5h-avg': -5,
        'price-6h-avg': -6,
        'price-8h-avg': -8,
        'price-10h-avg': -10,
        'price-12h-avg': -12,
        'price-24h-avg': -24,
        'price-48h-avg': -48,
        'price-96h-avg': -96,
        'price-192h-avg': -192,
        'price+24h-avg': 24
    },
    'volume': {
        'vol-0h-avg': 0,
        'vol-1h-avg': -1,
        'vol-2h-avg': -2,
        'vol-4h-avg': -4,
        'vol-6h-avg': -6,
        'vol-8h-avg': -8,
        'vol-10h-avg': -10,
        'vol-16h-avg': -16,
        'vol-24h-avg': -24,
        'vol-48h-avg': -48,
        'vol-96h-avg': -96,
        'vol-192h-avg': -192
    }
}

In [5]:
fields = [
    StructField("symbol", StringType(), False),
    StructField("date", TimestampType(), False),
]

for field, delta in DELTA_MATCH['price'].items():
    fields.append(StructField(field, DoubleType(), False))
for field, delta in DELTA_MATCH['volume'].items():
    fields.append(StructField(field, DoubleType(), False))    

schema = StructType(fields)

In [ ]:
T.transform_as_df(schema=schema, spark=spark, symbol="EOS", raw_df=df, sacrified_rows=192, past_delta_match=DELTA_MATCH)

In [ ]:
from pyspark.sql.window import Window
# window_01 = Window.partitionBy("customer_name").orderBy("date", "category").rowsBetween(-sys.maxsize, 0)
window = Window.rowsBetween(-1,0)
window_df = df_crypto.withColumn("price-1h", df_crypto['open'].over(window)).show()